# 1.2 Test (numerical) gradients

Differentiate through rollouts over different lengths. Answer the following questions:
-  How do gradients of $G$ compare to finite difference gradients of $F$? Make plots showing FD gradients of various perturbation sizes. Repeat this for increasing rollout time $T$, until both the FD and autograd gradients become numerically unstable.
- Compare FD and $G$-autograd gradients to symbolic gradients of $F$. As a test case, include a system with a closed form solution to its diffeqs, or better yet, to its discretized iterations. System of linear diffeqs seems to fit this. If that's too easy for a NN to learn, do a static invertible differentiable transformation of state space, so the NN has to learn nonlinear dynamics but we can still get accurate symbolic gradients.
- Discuss how For L96 true symbolic gradients may be high order polynomials, so $G$ might be `better' in the sense of giving a closer match to FDs when put into numerical practice.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import L96sim

from L96_emulator.util import dtype, dtype_np, device, as_tensor

res_dir = '/gpfs/work/nonnenma/results/emulators/L96/'
data_dir = '/gpfs/work/nonnenma/data/emulators/L96/'

### load / simulate some toy data
- note we're simulating with the numba simulator, so that one will potentially have an edge over the pytorch emulator in predicting it's own output

In [ ]:
from L96_emulator.util import predictor_corrector, rk4_default, get_data
from L96_emulator.run import sel_dataset_class

K, J, T, dt, N_trials = 40, 0, 605, 0.05, 1
spin_up_time, train_frac = 5., 0.8
normalize_data = False

F, h, b, c = 8., 1., 10., 10.

out, datagen_dict = get_data(K=K, J=J, T=T, dt=dt, N_trials=N_trials, F=F, h=h, b=b, c=c, 
                             resimulate=True, solver=rk4_default,
                             save_sim=False, data_dir=data_dir)

prediction_task = 'state'
lead_time = 1
DatasetClass = sel_dataset_class(prediction_task=prediction_task, N_trials=N_trials)
dg_train = DatasetClass(data=out, J=J, offset=lead_time, normalize=normalize_data, 
                   start=int(spin_up_time/dt), 
                   end=int(np.floor(T/dt*train_frac)))

plt.figure(figsize=(8,4))
if N_trials > 1:
    plt.imshow(out[0].T, aspect='auto')
else:
    plt.imshow(out.T, aspect='auto')
plt.xlabel('time')
plt.ylabel('location')
plt.show()

### finite differences
- scipy.optimize.approx_fprime

In [ ]:
from scipy.optimize import approx_fprime
from L96_emulator.util import predictor_corrector, rk4_default
import time


n_trials = 20 # number of parallel solves
n_starts = 5*int(spin_up_time/dt) * np.arange(1,n_trials+1) # initial conditions, taken from long simulation
n_rollout = 50 # rollout steps

from L96sim.L96_base import f1, pf2 # parallelized numba simulator

dX_dt = np.empty((K*(J+1), n_trials), dtype=dtype_np)
if J > 0:
    def fun(t, x):
        return pf2(x, F, h, b, c, dX_dt, K, J)
else:
    def fun(t, x):
        return f1(x, F, dX_dt, K)
             
times = dt * np.arange(0, n_rollout+1) # time points for numba solver, will only use np.diff(times)

def pred(x):
    return rk4_default(fun=fun, y0=x.copy().reshape(-1,n_trials), times=times)[-1,:]

def loss_np(x, target): 
    return ((pred(x)-target)**2).mean()

target = out[n_starts+n_rollout].T.copy() # predict after rollout
x = out[n_starts].T.copy()                # initialization estimate

# potentially corrupt initialization estimate (for non-zero gradients...)
eps = np.ones((x.shape))
eps[:K,:] *= np.mean(x[:K,:])
eps[K:,:] *= np.mean(x[K:,:])
x += np.random.normal(size=x.shape) * ( 0.1 * eps ) # corrupting with 0.1 std noise

def loss_sp(x):
    return loss_np(x, target=target)

# finite-difference gradients
t = time.time()
fprime_sp = approx_fprime(xk=x.reshape(-1), f=loss_sp, epsilon=1e-5).reshape(-1, len(n_starts))
print(f'took {time.time()-t}s')

plt.plot(fprime_sp)
plt.show()

# emulator gradients
- loss.backward()

In [ ]:
import torch 
import numpy as np
from L96_emulator.networks import named_network
from L96_emulator.eval import load_model_from_exp_conf, named_network
from L96_emulator.run import setup

exp_id = 26

if exp_id is None: 
    # loading 'perfect' (up to machine-precision-level quirks) L96 model in pytorch
    model, model_forwarder = named_network(
        model_name='MinimalConvNetL96',
        n_input_channels=J+1,
        n_output_channels=J+1,
        seq_length=1,
        **{'filters': [0],
           'kernel_sizes': [4],
           'init_net': 'analytical',
           'K_net': K,
           'J_net': J,
           'dt_net': 0.05,
           'model_forwarder': 'rk4_default'}
    )

else:
    # loading trained L96 model in pytorch
    exp_names = os.listdir('experiments/')   
    conf_exp = exp_names[np.where(np.array([name[:2] for name in exp_names])==str(exp_id))[0][0]][:-4]
    print('conf_exp', conf_exp)
    args = setup(conf_exp=f'experiments/{conf_exp}.yml')
    args.pop('conf_exp')

    args['model_forwarder'] = 'rk4_default'  # update numerical integration method to RK4
    args['dt_net'] = dt                      # with current step size

    model, model_forwarder, training_outputs = load_model_from_exp_conf(res_dir, args)

    if not training_outputs is None:
        training_loss, validation_loss = training_outputs['training_loss'], training_outputs['validation_loss']

        fig = plt.figure(figsize=(8,8))
        seq_length = args['seq_length']
        plt.semilogy(validation_loss, label=conf_exp+ f' ({seq_length * (J+1)}-dim)')
        plt.title('training')
        plt.ylabel('validation error')
        plt.legend()
        fig.patch.set_facecolor('xkcd:white')
        plt.show()

In [ ]:
import torch 
import numpy as np
from L96_emulator.eval import sortL96fromChannels, sortL96intoChannels, Rollout

def loss_torch(x, target):
    roller_outer.X = x    
    x_pred = roller_outer.forward()
    return ((x_pred - target)**2).mean()

# rollout operator
roller_outer = Rollout(model_forwarder, prediction_task='state', K=K, J=J, 
                       N=n_trials, T=n_rollout)

x_torch = as_tensor(sortL96intoChannels(np.atleast_2d(x.T),J=J))
x_torch = torch.nn.Parameter(x_torch)
target_torch = as_tensor(sortL96intoChannels(np.atleast_2d(target.T),J=J))

def loss_(x):
    return loss_torch(x, target=target_torch)

# torch loss
t = time.time()
loss = loss_(x_torch)
loss.backward()
grad_torch = sortL96fromChannels(roller_outer.X.grad.detach().cpu().numpy()).reshape(n_trials, -1).T
print(f'took {time.time()-t}s')

plt.plot(grad_torch)
plt.show()

## some plotting

In [ ]:
plt.figure(figsize=(16,24))
for i in range(n_trials):
    plt.subplot(np.ceil(n_trials/2), 2, i+1)
    plt.plot(grad_torch[:,i], label='torch')
    plt.plot(fprime_sp[:,i], '--', label='scipy fd')
plt.subplot(np.ceil(n_trials/2), 2, 1)
plt.legend()
plt.suptitle(f'rollout through {n_rollout} steps')
plt.show()


# share notebook results via html file

In [ ]:
!jupyter nbconvert --output-dir='/gpfs/home/nonnenma/projects/lab_coord/mdml_wiki/marcel/emulators' --to html numerical_gradients.ipynb